In [13]:
import os
from include.anchored_graph import AnchorGraph
from include.layer import Add, Concat
from include.function import rand_bool, rand_geo
from include.constants import WIDER_ADJUST, SKIP_PROB, SKIP_DECAY, SKIP_FEATURE
from include.model_trainer import Trainer

from fastai.vision import Learner, untar_data, URLs, ImageList, nn, accuracy, partial
from fastai.callbacks import EarlyStoppingCallback
from copy import deepcopy
import time

In [42]:
#export
import os
import time
from copy import deepcopy

from .anchored_graph import AnchorGraph
from .layer import Add, Concat
from .function import rand_bool, rand_geo
from .constants import WIDER_ADJUST, SKIP_PROB, SKIP_DECAY, SKIP_FEATURE
from .model_trainer import Trainer


ModuleNotFoundError: No module named '__main__.anchored_graph'; '__main__' is not a package

In [25]:
#export
class NetworkTransformTuner():
    def __init__(self, input_shape, output_shape, trainer:Trainer,
                 max_epoch=100, patience=3, min_delta=0):
        self.input_shape = input_shape
        self.output_shape = output_shape
        
        self.trainer = trainer
        self.trainer_args = {
            'max_epoch': max_epoch,
            'patience': patience,
            'min_delta': min_delta
        }
        
        self.path = self.trainer.path
        
        self.DAG = DAG(self, self.path)
        
    def fit(self, epoch):
        for _ in range(epoch):
            if self.DAG.length == 0:
                graph = AnchorGraph(self.input_shape, self.output_shape)
                model_num, accuracy = self.trainer.train(graph, **self.trainer_args)
                self.DAG.add_graph(model_id=model_num, father_id=0, accuracy=accuracy)
            else:
                graphs = self.DAG.get_best_graph(5)
                transforms = min(20//len(graphs), 4)
                
                for mid, _, graph in graphs:
                    for i in range(transforms):
                        past_historys = []
                        new_graph, history = transform(graph, deeper=True if i%2==0 else False)
                        if new_graph == None: continue
                        print('-'*40)
                        print(f'parent model:{mid}')
                        print(history_to_string(history))
                        model_num, accuracy = self.trainer.train(new_graph, **self.trainer_args)
                        self.DAG.add_graph(model_num, mid, accuracy)
    
    def fit_until(self, time_limit):
        start_time = time.time()
        elapsed = 0
        while elapsed < time_limit:
            if self.DAG.length == 0:
                graph = AnchorGraph(self.input_shape, self.output_shape)
                model_num, accuracy = self.trainer.train(graph, **self.trainer_args)
                self.DAG.add_graph(model_id=model_num, father_id=0, accuracy=accuracy)
            else:
                graphs = self.DAG.get_best_graph(5)
                elapsed = self._fit_until(graphs, start_time, time_limit)
    
    def _fit_until(self, graphs, start_time, time_limit):
        transforms = min(20//len(graphs), 4)
        for mid, _, graph in graphs:
            for i in range(transforms):
                new_graph, history = transform(graph, deeper=True if i%2==0 else False)
                if new_graph == None: continue
                print('-'*40)
                print(f'parent model:{mid}')
                print(history_to_string(history))
                model_num, accuracy = self.trainer.train(new_graph, **self.trainer_args)
                self.DAG.add_graph(model_num, mid, accuracy)
                elapsed = time.time() - start_time
                if(elapsed > time_limit): return elapsed;
        
        return time.time() - start_time
    
    def best_model(self):
        if self.DAG.length == 0:
            print('None of model detected.')
        else:
            return self.DAG.get_best_graph(1)

In [26]:
#export
class DAG():
    def __init__(self, caller, path):
        self.caller = caller
        self.path = path
        self.graphs = list()
    
    def add_graph(self, model_id, father_id, accuracy):
        new_graph = GraphContainer(model_id, father_id, accuracy)
        self.graphs.append(new_graph)
    
    def get_best_graph(self, max_graph=5):
        self.graphs.sort(key=lambda x: x.accuracy, reverse=True)
        graphs = self.graphs[:min(max_graph, len(self.graphs))] 
        return [(gc.model_id, gc.accuracy, self.retrieve_graph(gc)) for gc in graphs]
    
    def retrieve_graph(self, gr_container):
        graph = AnchorGraph(self.caller.input_shape, self.caller.output_shape)
        graph.load(os.path.join(self.path, str(gr_container.model_id)))
        return graph
    
    @property
    def length(self):
        return len(self.graphs)

In [27]:
#export
class GraphContainer():
    def __init__(self, model_id, father_id, accuracy):
        self._model_id = model_id
        self._father_id = father_id
        self._accuracy = accuracy
    
    @property
    def model_id(self):
        return self._model_id
    
    @property
    def father_id(self):
        return self._father_id
    
    @property
    def accuracy(self):
        return self._accuracy

In [28]:
#export
def transform(graph, deeper=False):
    new_graph = deepcopy(graph)
    convs = graph.convs
    anchor = new_graph.anchor
    max_anchor = sorted(anchor.keys())[-1]
    history = []
    
    # Deeper
    if deeper:
        new_graph.deeper_net(Add if rand_bool() else Concat)
        history.append({'method': 'deeper', 'target': None})
        
    # Wider & skip connection
    for id in convs:
        # wider random network.
        channel = new_graph.nodes[id].shape[0]
        prob = 1/(1 + channel/WIDER_ADJUST)
        if rand_bool(prob):
            new_graph.wider_net(id)
            history.append({'method': 'widen', 'target': id})
        
        # add connection to random anchor
        rank = new_graph.nodes[id].rank
        offset = rand_geo(p=0.5)
        anchor_rank = rank + offset
        if anchor_rank <= max_anchor and rand_bool(SKIP_PROB):
            num_layers = rand_geo(p=SKIP_DECAY)
            new_graph.add_connection(id, anchor[anchor_rank], 
                                     layer_features=[SKIP_FEATURE] * num_layers)
            history.append({'method': 'skip', 'target': (id, anchor[anchor_rank])})
        
    if len(history) == 0:
        return None, []
    else:
        return new_graph, history

In [29]:
#export
def history_to_string(history):
    ret = ''
    for item in history:
        if item['method'] == 'deeper':
            ret += (' - Deeper network\n')
        elif item['method'] == 'widen':
            ret += (' - Wider network on {}\n'.format(item['target']))
        elif item['method'] == 'skip':
            ret += (' - Skip connection on {} to {}\n'.format(*item['target']))
    return ret

In [43]:
!python nb2py.py search.ipynb

Converted search.ipynb to exp/nb_search.py


# Test

In [30]:
gr = AnchorGraph((1, 32, 32), (10,))
gr.deeper_net(Concat)
gr.wider_net(6)

In [31]:
gr = AnchorGraph((1, 32, 32), (10,))
gr.visualize('0', './random')
next = gr
for i in range(1, 10):
    print(f'model num: {i}')
    new, history = transform(next, deeper=rand_bool())
    if new == None:
        continue
    else:
        next = new
        print(history_to_string(history))
        new.visualize(str(i), './random')

model num: 1
 - Wider network on 6

model num: 2
model num: 3
 - Deeper network

model num: 4
model num: 5
 - Deeper network
 - Skip connection on 6 to 13

model num: 6
model num: 7
 - Wider network on 6
 - Skip connection on 6 to 17
 - Wider network on 18
 - Wider network on 24

model num: 8
 - Deeper network
 - Wider network on 24
 - Wider network on 28

model num: 9
 - Deeper network
 - Wider network on 28
 - Skip connection on 28 to 7



In [32]:
cifar_path = untar_data(URLs.CIFAR)
cifar_data = ImageList.from_folder(cifar_path).split_by_folder(train="train", valid="test").label_from_folder().databunch(bs=128)
trainer = Trainer(path='./search_test', data=cifar_data, loss_func=nn.CrossEntropyLoss())

In [33]:
nt = NetworkTransformTuner((3, 32, 32), (10,), trainer)

In [34]:
nt.fit_until(60*60)

epoch,train_loss,valid_loss,accuracy,time
0,1.907549,1.982775,0.256300,00:04
1,1.842360,1.814075,0.327800,00:04
2,1.820155,1.795588,0.333400,00:04
3,1.784519,1.689199,0.374500,00:04
4,1.759231,1.692428,0.387300,00:04
5,1.721674,1.694460,0.367200,00:04
6,1.713316,1.676627,0.383900,00:04
7,1.696265,1.692511,0.389000,00:04
8,1.678875,1.916854,0.316600,00:04
9,1.684774,1.604702,0.421600,00:04


Epoch 18: early stopping
Saving model 0... Done!
Model number: 0
Best accuracy: 0.44999998807907104
----------------------------------------
parent model:0
 - Deeper network
 - Wider network on 6



epoch,train_loss,valid_loss,accuracy,time
0,1.571870,1.855645,0.355900,00:04
1,1.500205,1.543958,0.455800,00:04
2,1.445725,1.832031,0.378000,00:04
3,1.420103,1.545803,0.432100,00:04
4,1.373609,1.454816,0.494500,00:04
5,1.341884,1.505319,0.492500,00:04
6,1.339016,1.672099,0.448400,00:04
7,1.311997,1.831463,0.436100,00:04
8,1.301375,1.193062,0.571700,00:04
9,1.273708,1.537461,0.476100,00:04


Epoch 16: early stopping
Saving model 1... Done!
Model number: 1
Best accuracy: 0.6189000010490417
----------------------------------------
parent model:0
 - Deeper network
 - Wider network on 6



epoch,train_loss,valid_loss,accuracy,time
0,1.582616,1.611928,0.425900,00:05
1,1.507589,1.808819,0.383400,00:05
2,1.465453,1.559836,0.451700,00:05
3,1.415387,1.545188,0.468900,00:05
4,1.380336,1.161012,0.590700,00:05
5,1.354823,1.437567,0.507800,00:05
6,1.337238,1.990049,0.412300,00:04
7,1.298989,1.167986,0.588800,00:04


...

Epoch 4: early stopping
Saving model 48... Done!
Model number: 48
Best accuracy: 0.7630000114440918
----------------------------------------
parent model:18
 - Deeper network
 - Wider network on 6
 - Wider network on 14
 - Skip connection on 14 to 13
 - Wider network on 24



epoch,train_loss,valid_loss,accuracy,time
0,0.851022,1.141723,0.694700,00:10
1,0.803381,0.880265,0.716400,00:10
2,0.730265,1.040574,0.696100,00:10
3,0.726913,2.147276,0.566500,00:10
4,0.699032,0.902018,0.713600,00:10


Epoch 5: early stopping
Saving model 49... Done!
Model number: 49
Best accuracy: 0.7164000272750854
----------------------------------------
parent model:18
 - Skip connection on 24 to 7



epoch,train_loss,valid_loss,accuracy,time
0,0.715863,1.199255,0.654400,00:07
1,0.699329,0.809891,0.750200,00:07
2,0.721639,0.914726,0.728000,00:08
3,0.683999,1.165885,0.690200,00:08
4,0.677442,1.106878,0.652500,00:07


Epoch 5: early stopping
Saving model 50... Done!
Model number: 50
Best accuracy: 0.7501999735832214


In [36]:
nt.best_model()

[(46,
  0.791100025177002,
  <include.anchored_graph.AnchorGraph at 0x7fbe9b0e2588>)]

In [41]:
nt.best_model()[0][2].visualize('best', './')